In [ ]:
!pip install deepxde --upgrade

In [ ]:
import deepxde as dde
import numpy as np

In [ ]:
def residual(x, y):    
    y = y[:, 0:1]
    dy_x = dde.grad.jacobian(y, x, i=0)
    return [dy_x - y]

In [ ]:
def boundary(_, on_initial):
    return on_initial

geom = dde.geometry.TimeDomain(0, 1)
ic = dde.icbc.IC(geom, lambda x: 1, boundary)

In [ ]:
def exact(x): # for verification
    return np.hstack((np.exp(x), ))

data = dde.data.PDE(geom, residual, [ic], num_boundary=2, solution=exact, num_domain=10, num_test=100)

In [ ]:
layer_size = [1] + [10] + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=0.1, metrics=["l2 relative error"])
losshistory, train_state = model.train(epochs=1000)

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)